In [1]:
import argparse
import logging
import math
import os
import random
from datetime import datetime
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, LoggingHandler, models, losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import json
from sentence_transformers.readers import InputExample

In [2]:
# Parse command line arguments
parser = argparse.ArgumentParser()
parser.add_argument("--model_name_or_path", type=str, default = "klue/roberta-base")
parser.add_argument("--max_seq_length", type=int, default=128)
parser.add_argument("--batch_size", type=int, default=8)
parser.add_argument("--num_epochs", type=int, default=5)
parser.add_argument("--output_dir", type=str, default="output")
parser.add_argument("--output_prefix", type=str, default="kor_sts_")
parser.add_argument("--seed", type=int, default=777)
args = parser.parse_args("")

In [3]:
# Fix random seed
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

# Configure logger
logging.basicConfig(
    format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S', level=logging.INFO, handlers=[LoggingHandler()]
)

# Read the dataset
model_save_path = os.path.join(
    args.output_dir,
    args.output_prefix + args.model_name_or_path.replace("/", "-") + '-' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)

In [4]:
# Define SentenceTransformer model
word_embedding_model = models.Transformer(args.model_name_or_path, max_seq_length=args.max_seq_length)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model], device = 'cuda')

Some weights of the model checkpoint at klue/roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for

In [5]:
def load_kor_sts_samples(filename):
    samples = []
    with open(filename,'r') as f:
        json_data = json.load(f)
    for row in json_data:
        score = float(row['labels']['label']) / 5.0  # Normalize score to range 0 ... 1
        samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))
    return samples

In [6]:
# Read the dataset
logging.info("Read KorSTS train/dev dataset")

train_file = './dataset/klue-sts-v1.1_train.json'
train_samples = load_kor_sts_samples(train_file)
random.shuffle(train_samples)

n_train = int(0.9 * len(train_samples))
valid_samples = train_samples[n_train:]
train_samples = train_samples[:n_train]

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=args.batch_size)
valid_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(valid_samples, batch_size=args.batch_size,
                                                                 name='sts-valid')

2022-03-14 12:28:27 - Read KorSTS train/dev dataset


In [7]:
train_loss = losses.CosineSimilarityLoss(model=model)

# Configure the training.
warmup_steps = math.ceil(len(train_dataloader) * args.num_epochs * 0.1)  # 10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

2022-03-14 12:28:29 - Warmup-steps: 657


In [8]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=valid_evaluator,
          epochs=args.num_epochs,
          show_progress_bar = False,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

/home/skymj1128/.conda/envs/wanted/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


2022-03-14 12:29:59 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset in epoch 0 after 1000 steps:
2022-03-14 12:30:02 - Cosine-Similarity :	Pearson: 0.9514	Spearman: 0.9065
2022-03-14 12:30:02 - Manhattan-Distance:	Pearson: 0.9463	Spearman: 0.9079
2022-03-14 12:30:02 - Euclidean-Distance:	Pearson: 0.9463	Spearman: 0.9079
2022-03-14 12:30:02 - Dot-Product-Similarity:	Pearson: 0.9391	Spearman: 0.8841
2022-03-14 12:30:02 - Save model to output/kor_sts_klue-roberta-base-2022-03-14_12-28-15
2022-03-14 12:30:29 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-valid dataset after epoch 0:
2022-03-14 12:30:32 - Cosine-Similarity :	Pearson: 0.9534	Spearman: 0.9108
2022-03-14 12:30:32 - Manhattan-Distance:	Pearson: 0.9479	Spearman: 0.9111
2022-03-14 12:30:32 - Euclidean-Distance:	Pearson: 0.9484	Spearman: 0.9115
2022-03-14 12:30:32 - Dot-Product-Similarity:	Pearson: 0.9366	Spearman: 0.8885
2022-03-14 12:30:32 - Save model to output/kor_sts_klue-roberta-base-20

In [9]:
# Load the stored model and evaluate its performance on STS benchmark dataset
model = SentenceTransformer(model_save_path)
logging.info("Read KorSTS benchmark test dataset")
test_file = './dataset/klue-sts-v1.1_dev.json'
test_samples = load_kor_sts_samples(test_file)
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
test_evaluator(model, output_path=model_save_path)

2022-03-14 12:39:18 - Load pretrained SentenceTransformer: output/kor_sts_klue-roberta-base-2022-03-14_12-28-15
2022-03-14 12:39:19 - Use pytorch device: cuda
2022-03-14 12:39:19 - Read KorSTS benchmark test dataset
2022-03-14 12:39:19 - EmbeddingSimilarityEvaluator: Evaluating the model on sts-test dataset:
2022-03-14 12:39:20 - Cosine-Similarity :	Pearson: 0.8897	Spearman: 0.8898
2022-03-14 12:39:20 - Manhattan-Distance:	Pearson: 0.8858	Spearman: 0.8850
2022-03-14 12:39:20 - Euclidean-Distance:	Pearson: 0.8864	Spearman: 0.8858
2022-03-14 12:39:20 - Dot-Product-Similarity:	Pearson: 0.8707	Spearman: 0.8702


0.8898365349320421